In [1]:
import torch, sys, glob, os, math
from pathlib import Path
import numpy as np
import torch.multiprocessing as mp
mp.set_start_method('fork')
from multiprocessing import Pool
from copy import deepcopy


/home/is/armin-sa/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Binarize Training Data

In [2]:
n = 16
cwd = os.getcwd()
prep_train_data_name = "ru_small_id.txt"


In [3]:
%cd data
!split -n l/{n} {prep_train_data_name} chunk_
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [4]:
def binarize_chunk(f_in_path):
    with open(f_in_path, "r") as f_in, open(
        "data/prep_" + Path(f_in_path).stem, "wb"
    ) as f_out:
        for line in f_in:
            arr = np.fromstring(line, dtype=np.int32, sep=" ")
            f_out.write(
                len(arr).to_bytes(2, byteorder=sys.byteorder, signed=False)
                + arr.tobytes()
            )


In [5]:
with Pool(processes=n) as p:
    p.map(binarize_chunk, glob.glob("data/chunk_*"))


In [6]:
%cd data
!rm chunk_*
!cat prep_chunk_* > ru_small.bin
!rm prep_chunk_*
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [7]:
sample_locs = [0]
with open("data/ru_small.bin", "rb") as f_in:
    while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
        arr = np.frombuffer(f_in.read(count * 4), count=count, dtype=np.int32)
        sample_locs.append(f_in.tell())

sample_locs = np.array(sample_locs[:-1])


In [8]:
# with open("data/ru_small.bin", "rb") as f:
#     f.seek(sample_locs[2])
#     count = int.from_bytes(f.read(2), byteorder=sys.byteorder, signed=False)
#     arr = np.frombuffer(f.read(count * 4), count=count, dtype=np.int32)


In [9]:
# def _gen():
#     with open("data/ru_small.bin", "rb") as f_in:
#         while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
#             arr = torch.frombuffer(f_in.read(count * 4), count=count, dtype=torch.int)
#             yield arr


In [10]:
# a =_gen()
# next(a)


In [15]:
class CC100DataSet(torch.utils.data.Dataset):
    def __init__(self, file_path, num_workers):
        super().__init__()
        self.sample_locs = deepcopy(sample_locs)
        self.file_handles = [open(file_path, "rb") for _ in range(num_workers)]
        self.file_handles.append(open(file_path, "rb"))

    def __len__(self):
        return len(self.sample_locs)

    def __getitem__(self, idx):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:
            worker_id = 0
        else:
            worker_id = worker_info.id
        self.file_handles[worker_id].seek(self.sample_locs[idx])
        count = int.from_bytes(
            self.file_handles[worker_id].read(2), byteorder=sys.byteorder, signed=False
        )
        # if count == 0:
        #     print(idx)
        arr = np.frombuffer(self.file_handles[worker_id].read(count * 4), count=count, dtype=np.int32)
        return arr


def collate_wrapper(batch):
    return batch


num_workers = 8
ds = CC100DataSet("data/ru_small.bin", num_workers)
# data_loader = torch.utils.data.DataLoader(ds, num_workers=0, batch_size=1, collate_fn=collate_wrapper)

# c=0
# for _ in data_loader:
#     c+=1


In [22]:
data_loader = torch.utils.data.DataLoader(
    ds,
    num_workers=num_workers,
    shuffle=False,
    batch_size=64,
    collate_fn=collate_wrapper,
    prefetch_factor=1,
    persistent_workers=True,
)

# x=iter(data_loader)
# print(next(x))
# x=iter(data_loader)
# print(next(x))

c = 0
l = []
for _ in data_loader:
    l.append(_)
    c+=1


In [20]:
# with open("data/ru_small.bin", "rb") as f:
#     f.seek(sample_locs[0])
#     count = int.from_bytes(f.read(2), byteorder=sys.byteorder, signed=False)
#     arr = np.frombuffer(f.read(count * 4), count=count, dtype=np.int32)
# arr

In [23]:
l[-1]

[array([    1, 23546,   130,  7009,   122,    15, 21192,    14,  2250,
           48,  6676,  4823,  4925,   639,    57,    59,     4,    21,
          180,  3702,    37,  6244,  9535,   102,  7729,     7,    14,
        25405, 11979,     5,   145,   340,     8, 22087,  2360,     7,
            8,   101,  9066,   904,  2360,    15,    10,   829,    55,
           48,    10,   141, 19281,     4,  5213,   566,    59,   307,
          135, 30948,   286,     7, 21680,   385,    23,    23,  2159,
         3000,   479,  2969,     4,  6013,     7, 11387, 15045,     5,
            2], dtype=int32),
 array([    1,   215,    21,    59,     8, 23809,     7, 14790,     4,
           98,   311,    49,  1997,   458,  3956,     7,    43,  2144,
          121,   105,    97,    14,  6300,  1543,  1994,    39,     2],
       dtype=int32),
 array([    1,   236,  7329,   228,     4, 22051,     7,   140,     9,
        10097,    76,     4,   405,     5,    79,  2607,     4, 19249,
         2554,     5,    

In [21]:
print(c)


15182
